In [113]:
import pandas as pd
import numpy as np
import math


df = pd.read_csv("joined_understorey_observations.csv")
df



Unnamed: 0  Year_monitoring  Plot_number Plot_treatment  Quadrat_number  \
0              0                6            1            Gap               1   
1              1                6            1            Gap               2   
2              2                6            1            Gap               3   
3              3                6            1            Gap               4   
4              4                3            1            Gap               4   
...          ...              ...          ...            ...             ...   
9628        9628                3           12            Gap              17   
9629        9629                6           12            Gap              17   
9630        9630                6           12            Gap              17   
9631        9631                3           12            Gap              21   
9632        9632                6           12            Gap              28   

      Quadrat_fenced Quadrat_gap_or_no  Record_ID T002_Flora_Species_name  \
0              False               NaN      10733          Stellaria spp.   
1              False               NaN      10988          Stellaria spp.   
2              False               NaN      11014          Stellaria spp.   
3              False               NaN      11034          Stellaria spp.   
4              False            In Gap        119          Stellaria spp.   
...              ...               ...        ...                     ...   
9628           False            In Gap       5462     Centella cordifolia   
9629           False               NaN      15622     Centella cordifolia   
9630           False               NaN      15631     Centella cordifolia   
9631           False        Not in Gap       5562            Acaena ovina   
9632            True               NaN      15800        Veronica plebeia   

      Score      Scientific Name              Common Name      Family Name  \
0       0.5      Stellaria sp. 1         Slender Starwort  Caryophyllaceae   
1       0.5      Stellaria sp. 1         Slender Starwort  Caryophyllaceae   
2       0.5      Stellaria sp. 1         Slender Starwort  Caryophyllaceae   
3       1.0      Stellaria sp. 1         Slender Starwort  Caryophyllaceae   
4       0.5      Stellaria sp. 1         Slender Starwort  Caryophyllaceae   
...     ...                  ...                      ...              ...   
9628    1.0  Centella cordifolia                 Centella         Apiaceae   
9629    0.5  Centella cordifolia                 Centella         Apiaceae   
9630    0.5  Centella cordifolia                 Centella         Apiaceae   
9631    0.5         Acaena ovina  Australian Sheep's Burr         Rosaceae   
9632    0.5     Veronica plebeia       Trailing Speedwell     Veronicaceae   

     Division Name         Life Form Origin  
0     Dicotyledons  Medium Forb/Herb    NaN  
1     Dicotyledons  Medium Forb/Herb    NaN  
2     Dicotyledons  Medium Forb/Herb    NaN  
3     Dicotyledons  Medium Forb/Herb    NaN  
4     Dicotyledons  Medium Forb/Herb    NaN  
...            ...               ...    ...  
9628  Dicotyledons  Medium Forb/Herb    NaN  
9629  Dicotyledons  Medium Forb/Herb    NaN  
9630  Dicotyledons  Medium Forb/Herb    NaN  
9631  Dicotyledons   Large Forb/Herb    NaN  
9632  Dicotyledons  Medium Forb/Herb    NaN  

[9633 rows x 16 columns]

# Data preparation

In [114]:
groups = df.groupby(["Plot_number", "Quadrat_number", "Year_monitoring"])

print(groups["Record_ID"].count())

print(groups["Scientific Name"].count())


# count for the ID for each quadrat
(groups["Record_ID"].count()).reset_index().to_csv('grouped.csv', index = False)

grouped_df = pd.read_csv("grouped.csv")
grouped_df


# count for the scientific name for each quadrat with its frequency
name_freq = []
name_count_list = []
for group in groups["Scientific Name"]:
    name_dic = {}
    for item in list(group[1]):
        if item in name_dic.keys():
            name_dic[item] += 1
        else:
            name_dic[item] = 1
    print(name_dic)
    name_count_list.append(len(name_dic.keys()))
    name_freq.append(name_dic)

# formulate the richness
grouped_df["Scientific_Name_count"] = name_count_list

# formulate the diverisity
diveristy = []
for item in name_freq:
    total = 0
    sumup = sum(item.values())
    for value in item.values():
        p = value/sumup
        total += p*math.log(p)
    diveristy.append(total)
grouped_df["Diversity"] = diveristy

# formulate the relative_abundance
grouped_df["Relative_abundance/Eveness"] = grouped_df["Diversity"]/ max(grouped_df["Diversity"])
grouped_df




Plot_number  Quadrat_number  Year_monitoring
1            1               0                  10
                             3                  15
                             6                  14
             2               0                  13
                             3                  11
                                                ..
12           31              3                  12
                             6                   8
             32              0                   7
                             3                   7
                             6                   4
Name: Record_ID, Length: 845, dtype: int64
Plot_number  Quadrat_number  Year_monitoring
1            1               0                  10
                             3                  15
                             6                  14
             2               0                  13
                             3                  11
                                                ..


Plot_number  Quadrat_number  Year_monitoring  Record_ID  \
0              1               1                0         10   
1              1               1                3         15   
2              1               1                6         14   
3              1               2                0         13   
4              1               2                3         11   
..           ...             ...              ...        ...   
840           12              31                3         12   
841           12              31                6          8   
842           12              32                0          7   
843           12              32                3          7   
844           12              32                6          4   

     Scientific_Name_count  Diversity  Relative_abundance/Eveness  
0                       10  -2.302585                    3.321928  
1                       15  -2.708050                    3.906891  
2                       13  -2.540036                    3.664498  
3                       13  -2.564949                    3.700440  
4                       11  -2.397895                    3.459432  
..                     ...        ...                         ...  
840                     12  -2.484907                    3.584963  
841                      8  -2.079442                    3.000000  
842                      7  -1.945910                    2.807355  
843                      7  -1.945910                    2.807355  
844                      4  -1.386294                    2.000000  

[845 rows x 7 columns]

# find shadow tolerant plant and calulate the relative abundance

#  Richness calcualtion

In [115]:
# calculate the absolute richess and relative richness
def find_abs_richness_dataframe(input_df, target):
    diff_list = []
    plot_num = []
    quadrat_num = []
    i = 0
    while i < len(grouped_df):
        if (grouped_df.iloc[i+1]["Plot_number"] == grouped_df.iloc[i]["Plot_number"]) and (grouped_df.iloc[i+1]["Quadrat_number"] == grouped_df.iloc[i]["Quadrat_number"]) and grouped_df.iloc[i+2]["Year_monitoring"] - grouped_df.iloc[i]["Year_monitoring"] == 6:
            plot_num.append(grouped_df.iloc[i]["Plot_number"])
            quadrat_num.append(grouped_df.iloc[i]["Quadrat_number"])
            diff = grouped_df.iloc[i+2][target] - grouped_df.iloc[i][target]
            diff_list.append(diff)
            i += 3
        else:
            plot_num.append(grouped_df.iloc[i]["Plot_number"])
            quadrat_num.append(grouped_df.iloc[i]["Quadrat_number"])
            diff = grouped_df.iloc[i+1][target]  - grouped_df.iloc[i][target] 
            diff_list.append(diff)
            i += 2
    result = pd.DataFrame({"Plot_number":plot_num, "Quadrat_number":quadrat_num, "abs_richness":diff_list})
    return result

# calucalte the absolute target value and structure them into an array
def find_abs_target_array(input_df, target):
    temp_list = []
    i = 0
    while i < len(grouped_df):
        if (grouped_df.iloc[i+1]["Plot_number"] == grouped_df.iloc[i]["Plot_number"]) and (grouped_df.iloc[i+1]["Quadrat_number"] == grouped_df.iloc[i]["Quadrat_number"]) and grouped_df.iloc[i+2]["Year_monitoring"] - grouped_df.iloc[i]["Year_monitoring"] == 6:
            diff = grouped_df.iloc[i+2]["Scientific_Name_count"] - grouped_df.iloc[i]["Scientific_Name_count"] 
            temp_list.append(round(diff,4))
            i += 3
        else:
            diff = grouped_df.iloc[i+1][target] - grouped_df.iloc[i][target] 
            temp_list.append(round(diff,4))
            i += 2
            
    return temp_list

# calucalte the relative target  value and structure them into an array
def find_relative_target_array(input_df, target):
    temp_list = []
    i = 0
    while i < len(grouped_df):
        if (grouped_df.iloc[i+1]["Plot_number"] == grouped_df.iloc[i]["Plot_number"]) and (grouped_df.iloc[i+1]["Quadrat_number"] == grouped_df.iloc[i]["Quadrat_number"]) and grouped_df.iloc[i+2]["Year_monitoring"] - grouped_df.iloc[i]["Year_monitoring"] == 6:
            diff = grouped_df.iloc[i+2]["Scientific_Name_count"] /grouped_df.iloc[i]["Scientific_Name_count"] 
            temp_list.append(round(diff,4))
            i += 3
        else:
            diff = grouped_df.iloc[i+1][target] /grouped_df.iloc[i][target] 
            temp_list.append(round(diff,4))
            i += 2
            
    return temp_list

result = find_abs_richness_dataframe(grouped_df,"Scientific_Name_count") 
result["relative_richness"] = find_relative_target_array(grouped_df, "Scientific_Name_count")
result["richness_change_rate"] = result["relative_richness"]-1
print(result)


     Plot_number  Quadrat_number  abs_richness  relative_richness  \
0            1.0             1.0           3.0             1.3000   
1            1.0             2.0           4.0             1.3077   
2            1.0             3.0           6.0             1.3750   
3            1.0             4.0           3.0             1.3333   
4            1.0             5.0           1.0             1.0714   
..           ...             ...           ...                ...   
283         12.0            28.0          -5.0             0.6154   
284         12.0            29.0          -2.0             0.8571   
285         12.0            30.0          -3.0             0.7500   
286         12.0            31.0           1.0             1.1429   
287         12.0            32.0          -3.0             0.5714   

     richness_change_rate  
0                  0.3000  
1                  0.3077  
2                  0.3750  
3                  0.3333  
4                  0.0714  
.. 

# Diversity calculation

In [120]:
result["abs_diversity"] = find_abs_target_array(grouped_df, "Diversity")
result["relative_diversity"] = find_relative_target_array(grouped_df, "Diversity")
result["diversity_change_rate"] = result["relative_diversity"]-1
print(result)

     Plot_number  Quadrat_number  abs_richness  relative_richness  \
0            1.0             1.0           3.0             1.3000   
1            1.0             2.0           4.0             1.3077   
2            1.0             3.0           6.0             1.3750   
3            1.0             4.0           3.0             1.3333   
4            1.0             5.0           1.0             1.0714   
..           ...             ...           ...                ...   
283         12.0            28.0          -5.0             0.6154   
284         12.0            29.0          -2.0             0.8571   
285         12.0            30.0          -3.0             0.7500   
286         12.0            31.0           1.0             1.1429   
287         12.0            32.0          -3.0             0.5714   

     richness_change_rate  relative_diversity  diversity_change_rate  \
0                  0.3000              1.3000                 0.3000   
1                  0.3077  

# Relative abundance calculation

In [119]:
result["abs_relative_abundance"] = find_abs_target_array(grouped_df, "Relative_abundance/Eveness")
result["relative_abundance"] = find_relative_target_array(grouped_df, "Relative_abundance/Eveness")
result["relative_abundance_change_rate"] = result["relative_abundance"]-1
print(result)

     Plot_number  Quadrat_number  abs_richness  relative_richness  \
0            1.0             1.0           3.0             1.3000   
1            1.0             2.0           4.0             1.3077   
2            1.0             3.0           6.0             1.3750   
3            1.0             4.0           3.0             1.3333   
4            1.0             5.0           1.0             1.0714   
..           ...             ...           ...                ...   
283         12.0            28.0          -5.0             0.6154   
284         12.0            29.0          -2.0             0.8571   
285         12.0            30.0          -3.0             0.7500   
286         12.0            31.0           1.0             1.1429   
287         12.0            32.0          -3.0             0.5714   

     richness_change_rate  relative_diversity  diversity_change_rate  \
0                  0.3000              1.3000                 0.3000   
1                  0.3077  

In [111]:
result.to_csv("understorey_output.csv")